<a href="https://colab.research.google.com/github/walnashgit/S29-QLORA-PHI2/blob/main/S29_QLORA_PHI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14

In [4]:
from datasets import load_dataset


# ds = load_dataset("OpenAssistant/oasst1")
dataset_name = "OpenAssistant/oasst1"

# train = ds['train']      # len(train)=84437 (95%)
# val = ds['validation']   # len(val)=4401 (5%)
raw_dataset = load_dataset(dataset_name, split="train")

In [5]:
from datasets import Dataset
count = -1
conversations = []
for item in raw_dataset:
    parent_id = item['parent_id']
    text = item['text']
    role = item['role']

    if parent_id is None:
        conversations.append('')
        count += 1


    if role == 'prompter':
        conversations[count] += '### Human: %s' % text
    elif role == 'assistant':
        conversations[count] += '### Assistant: %s' % text

#convo_list = [c for c in conversations]
dataset = Dataset.from_dict(dict(text=conversations))
print(dataset[5])

{'text': '### Human: Schreibe einen kurze und präzise Konstruktionsbeschreibung zu einem Dreieck ABC mit c=6\xa0cm, h_c=5\xa0cm und γ=40°. γ ist hierbei der von Seite c gegenüberliegende Winkel.### Assistant: Dreieck ABC ist ein rechtwinkliges Dreieck mit der Seitenlänge c=6 cm als Hypotenuse. Die Höhe h_c von c beträgt 5 cm und der Winkel γ von c gegenüberliegend beträgt 40°.### Human: Vielen Dank, das hat mir sehr weitergeholfen.### Assistant: Das Dreieck ABC ist ein rechtwinkliges Dreieck mit einer Seite c von 6 cm Länge und einer gegenüberliegenden Höhe h_c von 5 cm. Der Winkel γ, der gegenüberliegend zu Seite c ist, beträgt 40°.'}


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


# Load model directly
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [8]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "dense",
        "fc1",
    ]
)

In [9]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

In [10]:
from trl import SFTTrainer

max_seq_length = 256 #512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [11]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.909000
20,1.816400
30,1.755100
40,1.697700
50,1.813000
60,1.750800
70,1.741800
80,1.655900
90,1.727500
100,1.807300


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=500, training_loss=1.7165238761901855, metrics={'train_runtime': 4282.5669, 'train_samples_per_second': 1.868, 'train_steps_per_second': 0.117, 'total_flos': 3.265330249347072e+16, 'train_loss': 1.7165238761901855, 'epoch': 0.8123476848090982})

In [14]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

# Run text generation pipeline with our next model
prompt = "What is the future of AI?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"### Human: {prompt}\n\n### Assistant: ")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.

### Human: What is the future of AI?

### Assistant: 
The future of AI is a topic of much debate and speculation. Some experts predict that AI will continue to advance rapidly, leading to significant changes in many aspects of society, including the economy, healthcare, and education. Others are more cautious, warning of potential risks and ethical concerns associated with the development of AI.

One potential future of AI is the development of more advanced and sophisticated machine learning algorithms, which could lead to more accurate and efficient decision-making in a wide range of fields. This could include the development of AI-powered systems for healthcare, finance, and transportation, as well as the use of AI in fields such as education and entertainment.

Another potential future of AI is the development of more advanced natural language processing (NLP) systems, which could lead to more advanced communication and interaction between humans and machines. This could include th

In [15]:
!zip -r results.zip results/

  adding: results/ (stored 0%)
  adding: results/checkpoint-400/ (stored 0%)
  adding: results/checkpoint-400/special_tokens_map.json (deflated 75%)
  adding: results/checkpoint-400/tokenizer_config.json (deflated 94%)
  adding: results/checkpoint-400/vocab.json (deflated 59%)
  adding: results/checkpoint-400/tokenizer.json (deflated 72%)
  adding: results/checkpoint-400/adapter_model.safetensors (deflated 8%)
  adding: results/checkpoint-400/added_tokens.json (deflated 84%)
  adding: results/checkpoint-400/training_args.bin (deflated 51%)
  adding: results/checkpoint-400/rng_state.pth (deflated 25%)
  adding: results/checkpoint-400/merges.txt (deflated 53%)
  adding: results/checkpoint-400/adapter_config.json (deflated 52%)
  adding: results/checkpoint-400/scheduler.pt (deflated 57%)
  adding: results/checkpoint-400/README.md (deflated 66%)
  adding: results/checkpoint-400/trainer_state.json (deflated 78%)
  adding: results/checkpoint-400/optimizer.pt (deflated 9%)
  adding: results/c

In [17]:
!zip -r output.zip output/

  adding: output/ (stored 0%)
  adding: output/checkpoint-500/ (stored 0%)
  adding: output/checkpoint-500/special_tokens_map.json (deflated 75%)
  adding: output/checkpoint-500/tokenizer_config.json (deflated 94%)
  adding: output/checkpoint-500/vocab.json (deflated 59%)
  adding: output/checkpoint-500/tokenizer.json (deflated 72%)
  adding: output/checkpoint-500/adapter_model.safetensors (deflated 8%)
  adding: output/checkpoint-500/added_tokens.json (deflated 84%)
  adding: output/checkpoint-500/training_args.bin (deflated 51%)
  adding: output/checkpoint-500/rng_state.pth (deflated 25%)
  adding: output/checkpoint-500/merges.txt (deflated 53%)
  adding: output/checkpoint-500/adapter_config.json (deflated 52%)
  adding: output/checkpoint-500/scheduler.pt (deflated 57%)
  adding: output/checkpoint-500/README.md (deflated 66%)
  adding: output/checkpoint-500/trainer_state.json (deflated 79%)
  adding: output/checkpoint-500/optimizer.pt (deflated 9%)
  adding: output/runs/ (stored 0%)


In [16]:
from google.colab import files
files.download("results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>